In [ ]:
using JLD
include("Gync.jl");
parms, y0 = loadparms()
data = loadpfizer();

In [ ]:
SIGMA_RHO = 0.2
SIGMA_Y0  = 1
SIGMA_PARMS = 20
SIGMA_PROPOSAL = 0.1
adapt = :all
stepsize = 100
sampler = :amm
persons = 1
samplingnodes = [:sparms, :y0]
chains = 5


jldname = "$sampler $adapt ugm-flat $(length(persons))pX$(chains)c $(SIGMA_RHO)r $(SIGMA_Y0)sy $(SIGMA_PARMS)sp $(SIGMA_PROPOSAL)p.jld"

d = eye(length(SAMPLEPARMS)+length(y0))
proposalvariance = d * log(1+(SIGMA_PROPOSAL^2));

samplers = Dict(
    :amm=>[AMM(samplingnodes, proposalvariance, adapt=adapt)], 
    :amwg=>[AMWG(samplingnodes,diag(proposalvariance); target=0.44)])

mcs = []
for person in persons
    m, inp, ini = gyncmodel(data[person], parms, y0)
    setsamplers!(m, samplers[sampler])
    
    push!(mcs, mcmc(m, inp, [ini[1] for i=1:chains], stepsize, burnin=0, verbose = false, thin = 1, chains=chains))
end

#samplelength(mc) = max(floor(Int, size(mc, 1)/5), 2_000)

println(jldname)
while size(mcs[1],1) < 1_000_000
    @time for i in 1:length(persons)
        mcs[persons[i]] = mcmc(mcs[persons[i]], stepsize, verbose=false)
    end
    @time save(jldname, "mc", [mc.value for mc in mcs])    
    println(size(mcs[1]))
end

amm all ugm-flat 1pX5c 0.2r 1sy 20sp 0.1p.jld
 14.601944 seconds (3.09 M allocations: 573.897 MB, 0.38% gc time)
  0.020727 seconds (408 allocations: 21.188 KB)
(200,115,5)
 14.566795 seconds (3.12 M allocations: 657.765 MB, 0.44% gc time)
  0.096594 seconds (463 allocations: 22.703 KB)
(300,115,5)
 13.816204 seconds (3.12 M allocations: 658.253 MB, 0.46% gc time)
  0.035602 seconds (463 allocations: 22.703 KB)
(400,115,5)
 14.265361 seconds (3.13 M allocations: 658.755 MB, 0.45% gc time)
  0.052835 seconds (463 allocations: 22.703 KB)
(500,115,5)
 14.707902 seconds (3.12 M allocations: 659.176 MB, 0.44% gc time)
  0.053410 seconds (464 allocations: 22.719 KB)
(600,115,5)
 15.297164 seconds (3.13 M allocations: 659.644 MB, 0.43% gc time)
  0.076750 seconds (464 allocations: 22.719 KB)
(700,115,5)
 15.442771 seconds (3.13 M allocations: 660.184 MB, 0.45% gc time)
  0.048484 seconds (464 allocations: 22.719 KB)
(800,115,5)
 16.518951 seconds (3.11 M allocations: 660.263 MB, 0.44% gc time